# Day 15
## Puzzle 1

In [1]:
import numpy as np

In [2]:
input_file = 'input_1.txt'
# input_file = 'test_input_1.txt'
# input_file = 'test_input_2.txt'
# input_file = 'test_input_3.txt'

Read input warehouse map.

In [3]:
with open(file=input_file, mode="r") as file:
    map_lines = []
    file_content = file.read().split('\n\n')

    for line in file_content[0].split('\n'):
        map_lines.append(list(line.strip()))

    moves = list(file_content[1].replace(' ', '').replace('\n', ''))
    map_matrix = np.matrix(map_lines)

In [4]:
map_matrix

matrix([['#', '#', '#', ..., '#', '#', '#'],
        ['#', '.', '#', ..., '#', '.', '#'],
        ['#', '.', '.', ..., '.', 'O', '#'],
        ...,
        ['#', 'O', 'O', ..., '.', '.', '#'],
        ['#', 'O', '.', ..., 'O', 'O', '#'],
        ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

In [5]:
moves[:10]

['<', '>', '<', '>', '^', '<', '^', '^', 'v', 'v']

Extract starting position.

In [6]:
puzzle_1_map_matrix = map_matrix.copy()
starting_position = None
m, n = puzzle_1_map_matrix.shape

for i in range(m):
    for j in range(n):
        if puzzle_1_map_matrix[i, j] == '@':
            starting_position = np.array([i, j])

Iterate through the list of moves.

In [7]:
current_position = starting_position
directions = {'^': np.array([-1, 0]), '>': np.array([0, 1]), 'v': np.array([1, 0]), '<': np.array([0, -1])}

for move in moves:
    current_direction = directions[move]
    next_position = current_position + current_direction
    next_position_value = puzzle_1_map_matrix[*next_position]

    if next_position_value in ('.', '@'):  # Move to the next position if it is free.
        current_position = next_position

    elif next_position_value == 'O':  # If there is a box in the next position, we keep looking further and further behind the box until we find somthing else than a box (free space or wall).
        next_box_position = next_position + current_direction

        while puzzle_1_map_matrix[*next_box_position] == 'O':
            next_box_position += current_direction

        value_behind_boxes = puzzle_1_map_matrix[*next_box_position]

        if value_behind_boxes in ('.', '@'):  # If the space is free, we move the box from the next position to the free space and place ourself in the next position.
            puzzle_1_map_matrix[*next_box_position] = 'O'
            puzzle_1_map_matrix[*next_position] = '.'
            current_position = next_position

        else:
            continue
            
    else:
        continue

In [8]:
print(puzzle_1_map_matrix)

[['#' '#' '#' ... '#' '#' '#']
 ['#' 'O' '#' ... '#' '.' '#']
 ['#' 'O' 'O' ... 'O' 'O' '#']
 ...
 ['#' 'O' 'O' ... '.' '.' '#']
 ['#' 'O' 'O' ... 'O' 'O' '#']
 ['#' '#' '#' ... '#' '#' '#']]


Calculate the reusult according to the GPS coordinate system.

In [9]:
gps_coordinate_result = 0

for i in range(m):
    for j in range(n):
        if puzzle_1_map_matrix[i, j] == 'O':
            gps_coordinate_result += 100*i + j

In [10]:
gps_coordinate_result

1559280

## Puzzle 2

Make the map twice as wide.

In [11]:
updated_map = []

for i in range(m):
    line = []

    for j in range(n):
        if map_matrix[i, j] == '@':
            line.extend(['@', '.'])

        elif map_matrix[i, j] == 'O':
            line.extend(['[', ']'])

        else:
            line.extend([map_matrix[i, j]]*2)

    updated_map.append(line)

updated_map_matrix = np.matrix(updated_map)

In [12]:
print(updated_map_matrix)

[['#' '#' '#' ... '#' '#' '#']
 ['#' '#' '.' ... '.' '#' '#']
 ['#' '#' '.' ... ']' '#' '#']
 ...
 ['#' '#' '[' ... '.' '#' '#']
 ['#' '#' '[' ... ']' '#' '#']
 ['#' '#' '#' ... '#' '#' '#']]


Find the starting position in the wide map.

In [13]:
puzzle_2_map_matrix = updated_map_matrix.copy()
starting_position = None
m, n = puzzle_2_map_matrix.shape

for i in range(m):
    for j in range(n):
        if puzzle_2_map_matrix[i, j] == '@':
            starting_position = np.array([i, j])

Once again, iterate through the of moves.

In [14]:
current_position = starting_position
directions = {'^': np.array([-1, 0]), '>': np.array([0, 1]), 'v': np.array([1, 0]), '<': np.array([0, -1])}

for move in moves:
    current_direction = directions[move]
    next_position = current_position + current_direction
    next_position_value = puzzle_2_map_matrix[*next_position]

    if next_position_value in ('.', '@'):  # Same as before.
        current_position = next_position

    elif next_position_value in('[', ']'):
        if move == '>':  # If we are moving sideways, the process is basically the same as before.
            next_box_position = next_position + 2*current_direction
            box_move_counter = 2

            while puzzle_2_map_matrix[*next_box_position] == '[':
                next_box_position += 2*current_direction
                box_move_counter += 2

            value_behind_boxes = puzzle_2_map_matrix[*next_box_position]

            if value_behind_boxes in ('.', '@'):
                for box_move in range(1, box_move_counter + 1):
                    puzzle_2_map_matrix[*(next_position + current_direction*box_move)] = '[' if box_move % 2 == 1 else ']'

                puzzle_2_map_matrix[*next_position] = '.'
                current_position = next_position

            else:
                continue

        elif move == '<':
            next_box_position = next_position + 2*current_direction
            box_move_counter = 2

            while puzzle_2_map_matrix[*next_box_position] == ']':
                next_box_position += 2*current_direction
                box_move_counter += 2

            value_behind_boxes = puzzle_2_map_matrix[*next_box_position]

            if value_behind_boxes in ('.', '@'):
                for box_move in range(1, box_move_counter + 1):
                    puzzle_2_map_matrix[*(next_position + current_direction*box_move)] = ']' if box_move % 2 == 1 else '['

                puzzle_2_map_matrix[*next_position] = '.'
                current_position = next_position

        else:  # If we are moving up or down, we have to keep track of all the boxes that get affected and move them if there are spaces behind all boxes in the last step.
            affected_box_positions = []

            if next_position_value == '[':
                affected_box_positions.append([next_position, next_position + directions['>']])

            else:
                affected_box_positions.append([next_position, next_position + directions['<']])

            move_possible = False
            wall_encountered = False

            while not move_possible and not wall_encountered:
                affected_box_positions_to_add = set()
                all_positions_free = True

                for affected_box_position in affected_box_positions[-1]:
                    next_box_position = affected_box_position + current_direction
                    next_box_position_value = puzzle_2_map_matrix[*next_box_position]

                    if next_box_position_value == '[':
                        affected_box_positions_to_add |= {tuple(next_box_position), tuple(next_box_position + directions['>'])}
                        all_positions_free = False

                    elif next_box_position_value == ']':
                        affected_box_positions_to_add |= {tuple(next_box_position), tuple(next_box_position + directions['<'])}
                        all_positions_free = False

                    elif next_box_position_value == '#':
                        wall_encountered = True
                        all_positions_free = False
                        break

                    else:
                        continue

                if all_positions_free:
                    move_possible = True

                else:
                    affected_box_positions.append([np.array(val) for val in list(affected_box_positions_to_add)])

            if move_possible:
                for affected_box_position_row in affected_box_positions[::-1]:
                    for affected_box_position in affected_box_position_row:
                        puzzle_2_map_matrix[*(affected_box_position + current_direction)] = puzzle_2_map_matrix[*affected_box_position]
                        puzzle_2_map_matrix[*affected_box_position] = '.'

                current_position = next_position

    else:
        continue

In [15]:
print(puzzle_2_map_matrix)

[['#' '#' '#' ... '#' '#' '#']
 ['#' '#' '[' ... '.' '#' '#']
 ['#' '#' '[' ... ']' '#' '#']
 ...
 ['#' '#' '[' ... '.' '#' '#']
 ['#' '#' '[' ... ']' '#' '#']
 ['#' '#' '#' ... '#' '#' '#']]


In [16]:
gps_coordinate_result = 0

for i in range(m):
    for j in range(n):
        if puzzle_2_map_matrix[i, j] == '[':
            gps_coordinate_result += 100*i + j

In [17]:
gps_coordinate_result

1576353